In [4]:
import brian2
from brian2 import mV, ms, nS, mS, nF, Hz

import matplotlib.pyplot as plt

In [5]:
dt = 0.001*ms
inv_dt = 1./dt
update = inv_dt

N_e = 800
N_ir = 150
N_if = 50
N_all = N_e + N_ir + N_if

t_init = 500*inv_dt
t_pre = 0*inv_dt
t_stim = 500*inv_dt
t_post = 100*inv_dt
t_all = t_init + t_pre + t_stim + t_post

In [6]:
con_t, con_r, con_p = 0, 0, 0
import random
print(random.getstate())

(3, (2371430839, 3648991356, 2675527463, 3216447259, 3565807869, 484674022, 2214794467, 4169210388, 39897555, 739217273, 1161140022, 1291534646, 1516772389, 2563041944, 2972112270, 2722935939, 3021622918, 2403649551, 4065644086, 2065449647, 3491854776, 2574149604, 187114206, 2815893139, 3217760062, 3713138709, 1957854751, 2789219803, 3388242765, 3014840875, 319339418, 3765639582, 1770962851, 404355163, 2669799226, 2388228442, 2001218562, 497455888, 2315836768, 540518497, 4269832767, 3715116891, 3925947528, 1969384924, 2692653840, 2402308713, 3023926377, 2751570866, 1119245858, 414469429, 2956846455, 1119014247, 2344809677, 3994758580, 3937730695, 1082775216, 2955855564, 1072538303, 2928550757, 2923281764, 881147753, 897135304, 2460724196, 3635847135, 2201002995, 2842577226, 1923312232, 2896912930, 4294209531, 2441454156, 3878722215, 3947721549, 421421184, 4261417638, 2179427486, 1341068167, 2801752306, 2172227167, 3704696987, 1081448703, 2012901824, 4147218153, 3052497354, 3556548497, 

In [7]:
ampa_scl = 1.
nmda_e = 0.
nmda_rs = 0.
nmda_fs = 0.
nmda_scl_e = 0.45
nmda_scl_rs = 0.1
nmda_scl_fs = 0.1

gaba = 0
con_t = 0
con_p = 0
con_r = 0
con_if = 0
con_ir = 0
con_i_all = 0
i_del = 0
noise_wt = 0.016
W_scl = 0
decay_gaba = 5.

In [29]:
neuron_model = """
dv/dt = (v - V_L)/tau_m + I_syn/(1000000*uF) : volt
I_syn = I_AMPA + I_NMDA + I_GABA : ampere
"""

neuron_groups = {
    'PC': brian2.NeuronGroup(
        N_e, neuron_model,
        threshold='v > -52*mV',
        reset = 'v = -59*mV'
    ),
}

In [30]:
synapse_AMPA_model = """
dg_AMPA_r/dt = -g_AMPA_r/tau_AMPA_r : siemens (clock-driven)
dg_AMPA_d/dt = -g_AMPA_d/tau_AMPA_d : siemens (clock-driven)
g_AMPA = (g_AMPA_r - g_AMPA_d)/norm_AMPA : siemens
I_AMPA_post = -scl_AMPA*g_AMPA*(v_pre - V_E) : ampere (summed)
"""
synapse_AMPA_onpre = """
g_AMPA_r += w
g_AMPA_d += w
"""

synapse_NMDA_model = """
dg_NMDA_r/dt = -g_NMDA_r/tau_NMDA_r : siemens (clock-driven)
dg_NMDA_d/dt = -g_NMDA_d/tau_NMDA_d : siemens (clock-driven)
g_NMDA = (g_NMDA_r - g_NMDA_d)/norm_NMDA : siemens (clock-driven)
I_NMDA_post = -scl_NMDA*g_NMDA*(v_pre - V_E)/(1 + exp(-0.062*v_pre/3.57)) : ampere (summed)
"""
synapse_NMDA_onpre = """
g_NMDA_r += w
g_NMDA_d += w
"""

synapse_GABA_model = """
dg_GABA_r/dt = -g_GABA_r/tau_GABA_r : siemens (clock-driven)
dg_GABA_d/dt = -g_GABA_d/tau_GABA_d : siemens (clock-driven)
g_GABA = (g_GABA_r - g_GABA_d)/norm_GABA : siemens (clock-driven)
I_GABA_post = -scl_GABA*g_GABA*(v_pre - V_E) : ampere (summed)
"""
synapse_GABA_onpre = """
g_GABA_r += w
g_GABA_d += w
"""

In [31]:
S = brian2.Synapses(
    neuron_groups['PC'], target=neuron_groups['PC'], model=synapse_AMPA_model, on_pre=synapse_AMPA_onpre
)
#S.tau_AMPA_r=0.5*ms
#S.tau_AMPA_d=2.*ms
#S.norm_AMPA='0.472689 : 1 (constant over dt)'
#S.scl_AMPA='1. : 1 (constant over dt)'
#S.V_E='0.*mV : volt (constant over dt)'
#S.w='825*nS : siemens (constant over dt)'
S.connect(p=0.1, condition='i!=j')

ValueError: The summed variable "I_AMPA_post" does not referto any known variable in the target group.